In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

import nltk
import string

df = pd.read_pickle("./2019_cleaned.pkl")

In [2]:
df.head()

,date,screen_name,tweet,tweet_id,lang,cleaned_tweet
1,2020-10-04 01:16:24,shinyML,rl for chip design 🙌 everyone’s fave game #neu...,1312562178143248384,en,rl for chip design everyone’s fave game
2,2020-06-14 03:01:00,KirkDBorne,Analysis of #NeurIPS2019 papers by themes: htt...,1272001061054799872,en,analysis of papers by themes —————————— ...
3,2020-06-30 14:14:21,arXiv__ml,RT @Xingyu2017: How should we combine multiple...,1277968719969259520,en,rt how should we combine multiple auxiliary t...
4,2020-07-14 10:48:20,bguedj,"A 10-min video about our #NeurIPS2019 paper ""D...",1282990304929488896,en,a min video about our paper dichotomize and g...
6,2020-09-18 11:24:30,gorayni,#NeurIPS2019 Bibliometrics\n\nTop 5 most cited...,1306917005963005952,en,bibliometrics top most cited papers as toda...


In [3]:
corpus = df['cleaned_tweet']

In [4]:
vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")

Dimensions of Document-term matrix: (4718, 142308)


In [5]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english')
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")

Dimensions of Document-term matrix: (4718, 98707)


In [6]:
import gensim

In [7]:
# Tokenize the documents
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in corpus]

# Create a Gensim Dictionary.  This creates an id to word mapping for everything in our vocbulary
# It is NOT the same as the dictionary object in the Python standard library
mydict = gensim.corpora.Dictionary()

# Create a Gensim Corpus object.  This creates a list of tuples for each document.
# The first element of the tuple is the word id, the second is the number of counts
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_docs]

In [8]:
doc_term_matrix = gensim.matutils.corpus2dense(mycorpus, num_terms=len(mydict))

In [10]:
doc_term_matrix
tfidf = gensim.models.TfidfModel(mycorpus)
tfidf_matrix = gensim.matutils.corpus2dense(tfidf[mycorpus], num_terms=len(mydict))

In [11]:

tfidf_matrix

array([[0.46769   , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31106466, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.32098472, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.3217932 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.3217932 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.3217932 ]], dtype=float32)

In [12]:
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in corpus]

In [13]:
model = gensim.models.Word2Vec(tokenized_docs, size=10, window=2,min_count=1, sg=1)

In [14]:
import gensim.downloader as api
from gensim.test.utils import datapath

In [17]:
google_vec_file = './GoogleNews-vectors-negative300.bin'
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded
